In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import numpy as np

torch.set_default_device("cuda")

In [ ]:
# Define an Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()  # Sigmoid to constrain output between 0 and 1
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
def load_and_preprocess_obj(file_path):
    vertices = []

    try:
        with open(file_path, 'r') as obj_file:
            for line in obj_file:
                tokens = line.strip().split()
                if len(tokens) == 0:
                    continue

                # Process vertex data (assuming "v" lines in the OBJ file)
                if tokens[0] == 'v':
                    if len(tokens) >= 4:
                        # Extract the x, y, z coordinates
                        x = float(tokens[1])
                        y = float(tokens[2])
                        z = float(tokens[3])
                        vertices.append([x, y, z])

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

    if len(vertices) == 0:
        print("No vertex data found in the OBJ file.")
        return None

    # Convert the list of vertices to a NumPy array
    vertices = np.array(vertices, dtype=np.float32)

    return vertices

In [ ]:
# Training loop
def train_autoencoder(model, data, num_epochs=50, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs in data:
            optimizer.zero_grad()
            inputs = torch.tensor(inputs, dtype=torch.float32)
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

In [ ]:
# Load and preprocess your OBJ data
obj_data = load_and_preprocess_step("abc-dataset-obj/00030048_0ef34aa1b15748a5b4ad7c0e_trimesh_038.obj").T

# Define your input dimension and latent dimension
input_dim = len(obj_data[0])  # Modify based on your data structure
print(input_dim)
latent_dim = input_dim  # Adjust as needed

# Create and train the autoencoder
autoencoder = Autoencoder(input_dim, latent_dim)

train_autoencoder(autoencoder, obj_data)

# Reduce the number of meshes by encoding and decoding
def reduce_meshes(autoencoder, data):
    reduced_meshes = []
    for mesh in data:
        mesh = torch.tensor(mesh, dtype=torch.float32)
        encoded_mesh = autoencoder.encoder(mesh)
        decoded_mesh = autoencoder.decoder(encoded_mesh)
        reduced_meshes.append(decoded_mesh.detach().cpu().numpy())
    return reduced_meshes

# Use the trained autoencoder to reduce meshes
reduced_meshes = reduce_meshes(autoencoder, obj_data)

# Save or visualize the reduced meshes as needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ... (Previous code for Autoencoder, training, and mesh reduction)

# Function to visualize a 3D mesh
def visualize_mesh(mesh, title="Reduced Mesh"):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract x, y, z coordinates from the mesh
    x = mesh.loc[0]
    y = mesh.loc[1]
    z = mesh.loc[2]

    print(x)
    print(y)
    print(z)
    
    ax.scatter(x, y, z)
    ax.set_title(title)
    plt.show()

obj_data = pd.DataFrame(obj_data)
reduced_meshes = pd.DataFrame(reduced_meshes)

visualize_mesh(obj_data, title="Original Mesh")
visualize_mesh(reduced_meshes, title="Reduced Mesh")

# Reconstruct the mesh from the reduced version
reconstructed_mesh = autoencoder.decoder(torch.tensor(reduced_meshes.values, dtype=torch.float32)).detach().cpu().numpy()
reconstructed_mesh_df = pd.DataFrame(reconstructed_mesh)  # Convert the reconstructed mesh to a DataFrame
visualize_mesh(reconstructed_mesh_df, title="Reconstructed Mesh")
